In [11]:
import pandas as pd
from darts import TimeSeries
from darts.models import TFTModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import rmse, mae

# Load dataset for mandi price prediction
df = pd.read_excel(r"C:\Users\sshan\Desktop\DS\Project 4\Mandi price nhits.xlsx", sheet_name="nhits")

# Parse MM-YYYY column as datetime (first of month)
df["date"] = pd.to_datetime(df["MM-YYYY"], format="%m-%Y")

# Sort and check continuity for TFT modeling
df = df.sort_values("date").reset_index(drop=True)
print("Date range:", df["date"].min(), "to", df["date"].max())
print("Number of rows:", len(df))

# Check for missing months
expected_months = pd.date_range(df["date"].min(), df["date"].max(), freq="MS")
missing_months = expected_months.difference(df["date"])
print("Missing months:", missing_months)

# Set target variable
target_col = "Modal Price Avg(Rs./MT)"
series = TimeSeries.from_dataframe(df, time_col="date", value_cols=target_col, freq="MS")

# Set Covariates (all other features)
covariate_cols = [c for c in df.columns if c not in ["date", "Month", "Year", target_col, "MM-YYYY"]]
covariates = TimeSeries.from_dataframe(df, time_col="date", value_cols=covariate_cols, freq="MS")

# Scaling
scaler_target = Scaler()
scaler_cov = Scaler()

series_scaled = scaler_target.fit_transform(series)
covariates_scaled = scaler_cov.fit_transform(covariates)

# Train/val split
train, val = series_scaled[:-11], series_scaled[-11:-5]
cov_train, cov_val = covariates_scaled[:-11], covariates_scaled[-11:-5]

# Initialize TFT
model = TFTModel(
    input_chunk_length=12,      # look back 12 months
    output_chunk_length=6,      # forecast 6 months
    hidden_size=32,             # model size (tuneable)
    lstm_layers=1,              # number of LSTM layers
    num_attention_heads=4,      # multi-head attention
    dropout=0.1,
    batch_size=32,
    n_epochs=200,
    add_relative_index=True,    # helps with temporal position encoding
    random_state=42
)

# Fit model
model.fit(
    series=train,
    past_covariates=covariates_scaled, # use your scaled covariates as past inputs
    future_covariates=covariates_scaled,
    verbose=True
)

# Forecast next 6 months
forecast = model.predict(n=6, past_covariates=covariates_scaled, future_covariates=covariates_scaled)

# Evaluate
print("RMSE:", rmse(val, forecast))
print("MAE:", mae(val, forecast))

# Inverse transform for actual values
forecast_real = scaler_target.inverse_transform(forecast)
val_real = scaler_target.inverse_transform(val)

print("Validation actual vs forecast:")
print(pd.DataFrame({
    "actual": val_real.values().flatten(),
    "forecast": forecast_real.values().flatten()
}, index=val_real.time_index))

Date range: 2019-04-01 00:00:00 to 2025-08-01 00:00:00
Number of rows: 77
Missing months: DatetimeIndex([], dtype='datetime64[ns]', freq='MS')


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

   | Name                              | Type                             | Params
----------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0     
1  | val_metrics                       | MetricCollection                 | 0     
2  | input_embeddings                  | _MultiEmbedding                  | 0     
3  | static_covariates_vsn             | _VariableSelectionNetwork        | 0     
4  | encoder_vsn                       | _VariableSelectionNetwork        | 63.3 K
5  | decoder_vsn                       | _VariableSelectionNetwork        | 30.3 K
6  | static_context_grn                | _GatedResidualNetwork            | 4.3 K 
7  | static_context_hidden_encoder_grn | _GatedResidualNetwork            | 4.3 K 
8  | static_cont

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.07367524734608594
MAE: 0.06088761979157916
Validation actual vs forecast:
                  actual      forecast
date                                  
2024-10-01  33780.805687  36849.437993
2024-11-01  39628.553971  39287.683171
2024-12-01  38958.850575  39474.103135
2025-01-01  36224.605809  37138.162822
2025-02-01  35315.397490  38487.776680
2025-03-01  34637.175141  37071.444474
